In [1]:
import uproot, pylhe, glob, os
import numpy as np
import numba,vector
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd 
import seaborn as sns

In [2]:
lhe_signal=glob.glob("./Samples/Signal/SM/WWW01j_000[0-4].lhe.gz")
root_signal=glob.glob("./Samples/Signal/SM/WWW01j_000[0-4].root")

lhe_bkg=glob.glob("./Samples/3l01j/3l01j_00[0-1]?.lhe.gz")
root_bkg=glob.glob("./Samples/3l01j/3l01j_00[0-1]?.root")

print(root_bkg)

['./Samples/3l01j/3l01j_0019.root', './Samples/3l01j/3l01j_0015.root', './Samples/3l01j/3l01j_0003.root', './Samples/3l01j/3l01j_0002.root', './Samples/3l01j/3l01j_0014.root', './Samples/3l01j/3l01j_0018.root', './Samples/3l01j/3l01j_0013.root', './Samples/3l01j/3l01j_0005.root', './Samples/3l01j/3l01j_0009.root', './Samples/3l01j/3l01j_0008.root', './Samples/3l01j/3l01j_0004.root', './Samples/3l01j/3l01j_0012.root', './Samples/3l01j/3l01j_0007.root', './Samples/3l01j/3l01j_0011.root', './Samples/3l01j/3l01j_0010.root', './Samples/3l01j/3l01j_0006.root', './Samples/3l01j/3l01j_0001.root', './Samples/3l01j/3l01j_0017.root', './Samples/3l01j/3l01j_0016.root', './Samples/3l01j/3l01j_0000.root']


In [3]:
def get_xSection(lhefiles):
    init=pylhe.read_lhe_init(lhefiles[0])

    xSection=0.
    for process in init['procInfo']:
        xSection+=process['xSection']
    return xSection # in pb

In [4]:
def getNeventsLHE(lhefiles):
    N=0
    for f in lhefiles:
        lines=os.popen('zgrep "</event>" '+f+"|wc -l").readlines()
        N+=int(lines[0])
    return N

In [5]:
xSection_sig=get_xSection(lhe_signal)
xSection_bkg=get_xSection(lhe_bkg)
print(xSection_sig,xSection_bkg)

0.017251415 0.8911546000000001


In [6]:
N_lhe_sig=1000000 #getNeventsLHE(lhe_signal)
N_lhe_bkg=4000000 #getNeventsLHE(lhe_bkg)
print(N_lhe_sig,N_lhe_bkg)

1000000 4000000


In [7]:
def getNeventsRoot(rootfiles):
    N=0
    for f in rootfiles:
        with uproot.open(f+':Delphes') as tree:
            N+=tree.num_entries
    return N

In [8]:
N_root_sig=511723 #getNeventsRoot(root_signal)
N_root_bgk=2964674 #getNeventsRoot(root_bkg)
print(N_root_sig,N_root_bgk)

511723 2964674


In [9]:
#calculate resulting xSection*efficiency
xSection_sig*=N_root_sig/N_lhe_sig
xSection_bkg*=N_root_bgk/N_lhe_bkg
print(xSection_sig,xSection_bkg)

0.008827945838045 0.6604957181501001


In [10]:
#show available branches
tree=uproot.open(root_signal[0]+':Delphes')
tree.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
Event                | int32_t                  | AsDtype('>i4')
Event/Event.fUniq... | uint32_t[]               | AsJagged(AsDtype('>u4'))
Event/Event.fBits    | uint32_t[]               | AsJagged(AsDtype('>u4'))
Event/Event.Number   | int64_t[]                | AsJagged(AsDtype('>i8'))
Event/Event.ReadTime | float[]                  | AsJagged(AsDtype('>f4'))
Event/Event.ProcTime | float[]                  | AsJagged(AsDtype('>f4'))
Event/Event.Proce... | int32_t[]                | AsJagged(AsDtype('>i4'))
Event/Event.MPI      | int32_t[]                | AsJagged(AsDtype('>i4'))
Event/Event.Weight   | float[]                  | AsJagged(AsDtype('>f4'))
Event/Event.Cross... | float[]                  | AsJagged(AsDtype('>f4'))
Event/Event.Cross... | float[]                  | AsJagged(AsDtype('>f4'))
Event/Event.Scale    | 

In [11]:
#show only specifc elements
tree['MissingET'].show(filter_name="MissingET.*")

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
MissingET            | int32_t                  | AsDtype('>i4')                
MissingET.fUniqueID  | uint32_t[]               | AsJagged(AsDtype('>u4'))
MissingET.fBits      | uint32_t[]               | AsJagged(AsDtype('>u4'))
MissingET.MET        | float[]                  | AsJagged(AsDtype('>f4'))
MissingET.Eta        | float[]                  | AsJagged(AsDtype('>f4'))
MissingET.Phi        | float[]                  | AsJagged(AsDtype('>f4'))


In [12]:

tree['Electron'].show(filter_name="Electron.*", name_width=40)

name                                     | typename                 | interpretation                
-----------------------------------------+--------------------------+-------------------------------
Electron                                 | int32_t                  | AsDtype('>i4')                
Electron.fUniqueID                       | uint32_t[]               | AsJagged(AsDtype('>u4'))
Electron.fBits                           | uint32_t[]               | AsJagged(AsDtype('>u4'))
Electron.PT                              | float[]                  | AsJagged(AsDtype('>f4'))
Electron.Eta                             | float[]                  | AsJagged(AsDtype('>f4'))
Electron.Phi                             | float[]                  | AsJagged(AsDtype('>f4'))
Electron.T                               | float[]                  | AsJagged(AsDtype('>f4'))
Electron.Charge                          | int32_t[]                | AsJagged(AsDtype('>i4'))
Electron.EhadOverEem            

In [13]:
#specify the variables we are interested in
tree.keys(filter_name="/(MissingET.(MET|Phi)|Particle.(PID|Status)|Electron.(PT|Eta|Phi|Charge)|"
          "Muon.(PT|Eta|Phi|Charge)|Jet.(PT|Eta|Phi))/")

['Particle/Particle.PID',
 'Particle/Particle.Status',
 'Jet/Jet.PT',
 'Jet/Jet.Eta',
 'Jet/Jet.Phi',
 'Jet/Jet.PTD',
 'Electron/Electron.PT',
 'Electron/Electron.Eta',
 'Electron/Electron.Phi',
 'Electron/Electron.Charge',
 'Muon/Muon.PT',
 'Muon/Muon.Eta',
 'Muon/Muon.Phi',
 'Muon/Muon.Charge',
 'MissingET/MissingET.MET',
 'MissingET/MissingET.Phi']

In [14]:
@numba.jit(nopython=True)
def checkHiggs(batch):
    higgs=np.full((len(batch),),False)
    for i in range(len(batch)):
        for p in range(len(batch[i]['Particle.PID'])):
            if abs(batch[i]['Particle.PID'][p])==25:
                higgs[i]=True
                break
    return higgs

In [15]:
@numba.jit(nopython=True)
def preselection(batch):
    pass_selection=np.full((len(batch),),False)
    for i in range(len(batch)):
        nElectrons=0
        nMuons=0
        nJets=0

        for e in range(len(batch[i]['Electron.PT'])):
            if batch[i]['Electron.PT'][e]>=10.0 and abs(batch[i]['Electron.Eta'][e])<2.5:
                nElectrons+=1


        for m in range(len(batch[i]['Muon.PT'])):
            if batch[i]['Muon.PT'][m]>=10.0 and abs(batch[i]['Muon.Eta'][m])<2.5:
                nMuons+=1

        for j in range(len(batch[i]['Jet.PT'])):
            if batch[i]['Jet.PT'][j]>=25.0 and abs(batch[i]['Jet.Eta'][j])<2.5:
                nJets+=1

        if nElectrons+nMuons==3 and nJets<5:
            pass_selection[i]=True

    return pass_selection
    

In [16]:
@numba.jit(nopython=True)
def get_MET(batch):
    met=np.zeros((len(batch),))
    metPhi=np.zeros((len(batch),))
    for i in range(len(batch)):
        met[i]=float(batch[i]['MissingET.MET'][0])
        metPhi[i]=float(batch[i]['MissingET.Phi'][0])
    return met, metPhi

In [23]:
def find_unique_charge(batch):
    for i in range(len(batch)):
        charge = []
        leptons = []
        lepton0 = []
        for e in range(len(batch[i]['Electron.Charge'])):
            charge.append([batch[i]['Electron.Charge'][e], vector.obj(pt=batch[i]['Electron.PT'][e], 
                                      phi=batch[i]['Electron.Phi'][e], 
                                      eta=batch[i]['Electron.Eta'][e], 
                                      mass=511./1e6)])
        for m in range(len(batch[i]['Muon.Charge'])):
            charge.append([batch[i]['Muon.Charge'][m], vector.obj(pt=batch[i]['Muon.PT'][m], 
                                      phi=batch[i]['Muon.Phi'][m], 
                                      eta=batch[i]['Muon.Eta'][m], 
                                      mass=105.66/1e3)]) 
        if charge[0][0] == charge[1][0]:
            lepton0.append(charge[2][1])
        elif charge[0][0] == charge[2][0]:
            lepton0.append(charge[1][1])
        else:
            lepton0.append(charge[0][1])
        
    return lepton0


In [18]:
#selects which data type we are putting through the processing
def batch_selector(batch, decay):
    higgs = checkHiggs(batch)
    presel = preselection(batch)
    if decay == 0:
        batch = batch[np.logical_and(higgs,presel)]
    elif decay == 1:
        batch = batch[np.logical_and(~higgs,presel)]
    else:
        batch = batch[presel]
    
    return batch

#def batch_selector(batch):
 #   higgs = checkHiggs(batch)
  #  presel = preselection(batch)
   # batch = batch[np.logical_and(higgs,presel)] 
    #return batch

In [19]:
#get l0, l1, l2 where l0 has unique charge, l1 is closest to it and l2 is the other
def get_leptons(batch):
    DRll=np.full((len(batch),),1000.)
    lepton0s = find_unique_charge(batch)
    lepton1s = []
    lepton2s = []
    leptons=[]
    for i in range(len(batch)):
    
        #loop over electrons
        for e in range(len(batch[i]['Electron.PT'])):
            leptons.append(vector.obj(pt=batch[i]['Electron.PT'][e], 
                                      phi=batch[i]['Electron.Phi'][e], 
                                      eta=batch[i]['Electron.Eta'][e], 
                                      mass=511./1e6))
        #loop over muons
        for m in range(len(batch[i]['Muon.PT'])):
            leptons.append(vector.obj(pt=batch[i]['Muon.PT'][m], 
                                      phi=batch[i]['Muon.Phi'][m], 
                                      eta=batch[i]['Muon.Eta'][m], 
                                      mass=105.66/1e3)) # in GeV
           
             
        mindr=DRll[i]
        for l in leptons:
            for l0 in lepton0s:
                if l==l0:
                    continue
                if l.deltaR(l0)<mindr:
                    mindr=l.deltaR(l0)
                    lepton1s.append(l)
        for l in leptons:
            for l0 in lepton0s:
                for l1 in lepton0s:
                    if l==l0:
                        continue
                    elif l==l1:
                        continue
                    else:
                        lepton2s.append(l)
    
    return lepton0s, lepton1s, lepton2s
                

In [31]:
def get_data(batch, lepton0s, lepton1s, lepton2s):
    for i in range(len(batch)):
        l0 = lepton0s[0]
        l1 = lepton1s[i]
        l2 = lepton2s[i]

        PT_l0 = l0.p
        PT_l1 = l1.p
        PT_l2 = l2.p
        PT = np.array([PT_l0, PT_l1, PT_l2])

        #deltaPhi_l0_MET = l1.deltaangle(METvec)
       # deltaPhi_l1_MET = l1.deltaangle(METvec)
        #deltaPhi_l2_MET = l2.deltaangle(METvec)

        invarM_l0l2=(l0.p2+l2.p2)**2
        invarM_l1l2=(l1.p2+l2.p2)**2
        invarM_l0l1=(l0.p2+l1.p2)**2
        invarM_pairs = np.array([invarM_l0l2, invarM_l0l1, invarM_l1l2])

        invarM=(l1.p2+l2.p2+l0.p2)**2

        E_l1l2=np.sqrt((l1.p+l2.p)**2 + invarM_l1l2)
        E_l0l2=np.sqrt((l0.p+l2.p)**2 + invarM_l0l2)
        E_l0l1=np.sqrt((l0.p+l1.p)**2 + invarM_l0l1)
        transM_l1l2=np.sqrt((E_l1l2+batch[i]['MissingET.MET'][0])**2 - np.abs(l1.p * l2.p + batch[i]['MissingET.MET'][0])**2)
        transM_l0l2=np.sqrt((E_l0l2+batch[i]['MissingET.MET'][0])**2 - np.abs(l0.p * l2.p + batch[i]['MissingET.MET'][0])**2)
        transM_l0l1=np.sqrt((E_l0l1+batch[i]['MissingET.MET'][0])**2 - np.abs(l0.p * l1.p + batch[i]['MissingET.MET'][0])**2)
        transM_pairs=np.array([transM_l1l2, transM_l0l2, transM_l0l1])

        deltaeta_l1l2=l1.deltaeta(l2)
        deltaeta_l0l2=l0.deltaeta(l2)
        deltaeta_l0l1=l0.deltaeta(l1)
        deltaeta_pairs = np.array([deltaeta_l0l1, deltaeta_l0l2, deltaeta_l1l2])

        deltaangle_l1l2=l1.deltaangle(l2)
        deltaangle_l0l2=l0.deltaangle(l2)
        deltaangle_l0l1=l0.deltaangle(l1) 
        deltaangle_pairs = np.array([deltaangle_l0l1, deltaangle_l0l2, deltaangle_l1l2])

        dR_l1l2 = l1.deltaR(l2)
        dR_l0l1 = l0.deltaR(l1)
        dR_l0l2 = l0.deltaR(l2)
        dR_pairs = np.array([dR_l1l2, dR_l0l1, dR_l0l2])

        totalP = l0.p + l1.p + l2.p 


    return PT_l0, PT_l1, PT_l2, invarM_l0l1, invarM_l0l2, invarM_l1l2, invarM, \
                transM_l0l1, transM_l0l2, transM_l1l2, \
                deltaeta_l0l1, deltaeta_l0l2, deltaeta_l1l2, \
                deltaangle_l0l1, deltaangle_l0l2, deltaangle_l1l2, \
                dR_l0l1, dR_l0l2, dR_l1l2, totalP
    

In [28]:
@numba.jit(nopython=True)
def get_minDRll(batch):  
    
    batch = batch_selector(batch) #makes batch only have type as passed from selection criteria
                                            #this means there are 3 leptons        
    lepton0s, lepton1s, lepton2s = get_leptons(batch)
    
    get_data(batch, lepton0s, lepton1s, lepton2s)

    return lepton0s, lepton1s, lepton2s

In [29]:
myfilter="/(MissingET.(MET|Phi)|Particle.(PID|Status)|Electron.(PT|Eta|Phi|Charge)|Muon.(PT|Eta|Phi|Charge)|Jet.(PT|Eta|Phi))/"
batch_size=1000
iterator = uproot.iterate([f+":Delphes" for f in root_signal],step_size=batch_size, filter_name=myfilter)
for batch in tqdm(iterator, total=N_root_sig//batch_size):
    batch = batch_selector(batch, 0)
    l0s, l1s, l2s = get_leptons(batch)
    print("batch done")
    

  0%|                                                   | 0/511 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
print(l0s)

In [34]:
ntotal_VH=0
nselected_VH=0
ntotal_WWW=0
nselected_WWW=0
df_VH = None

myfilter="/(MissingET.(MET|Phi)|Particle.(PID|Status)|Electron.(PT|Eta|Phi|Charge)|Muon.(PT|Eta|Phi|Charge)|Jet.(PT|Eta|Phi))/"
batch_size=1000
iterator = uproot.iterate([f+":Delphes" for f in root_signal],step_size=batch_size, filter_name=myfilter)

for batch in tqdm(iterator, total=N_root_sig//batch_size):
    batch = batch_selector(batch, 0) #VH data
    higgs = checkHiggs(batch)
    presel = preselection(batch)
    met, metPhi = get_MET(batch)
    l0s, l1s, l2s = get_leptons(batch)
    PT_l0, PT_l1, PT_l2, invarM_l0l1, invarM_l0l2, invarM_l1l2, invarM, \
            transM_l0l1, transM_l0l2, transM_l1l2, \
            deltaeta_l0l1, deltaeta_l0l2, deltaeta_l1l2, \
            deltaangle_l0l1, deltaangle_l0l2, deltaangle_l1l2, \
            dR_l0l1, dR_l0l2, dR_l1l2, totalP = get_data(batch, l0s, l1s, l2s)
    
    dataset = {'PT_l0': PT_l0, 'PT_l1': PT_l1, 'PT_l2': PT_l2, 'met': met, 'metPhi': metPhi,
       'm_l0l1': invarM_l0l1, 'm_l0l2': invarM_l0l2, 'm_l1l2': invarM_l1l2, 'm_lll': invarM,
       'mT_l0l1': transM_l0l1, 'mT_l0l2': transM_l0l2, 'mT_l1l2': transM_l1l2, 
       'delEta_l0l1': deltaeta_l0l1, 'delEta_l0l2': deltaeta_l0l2, 'delEta_l1l2': deltaeta_l1l2,
      'delPhi_l0l1': deltaangle_l0l1, 'delPhi_l0l2': deltaangle_l0l2, 'delPhi_l1l2': deltaangle_l1l2,
      'delR_l0l1': dR_l0l1, 'delR_l0l2': dR_l0l2, 'delR_l1l2': dR_l1l2, 
        'sumPT': totalP} #make into dict
    
    
    if df_VH is not None:
            newdata_VH = pd.DataFrame(dataset)
            df_VH = pd.concat([df_VH, newdata_VH], axis = 0, ignore_index=True)
            
    else:
        df_VH = pd.DataFrame(dataset)
        
        ntotal_VH+=higgs.sum()
        ntotal_WWW+=(~higgs).sum()
        nselected_VH+=np.logical_and(higgs,presel).sum()
        nselected_WWW+=np.logical_and(~higgs,presel).sum()
                  
#print("Preselection efficiency for VH events: %2.2f %%"%(100.*nselected_VH/ntotal_VH))
#print("Preselection efficiency for WWW events: %2.2f %%"%(100.*nselected_WWW/ntotal_WWW))


515it [2:08:42, 15.00s/it]                                                      


In [54]:
print(l2s)

[MomentumObject4D(pt=24.532987594604492, phi=-3.0067577362060547, eta=0.5549740195274353, mass=0.000511), MomentumObject4D(pt=33.686851501464844, phi=-1.9392164945602417, eta=0.13847164809703827, mass=0.10565999999999999), MomentumObject4D(pt=14.596811294555664, phi=2.0501394271850586, eta=0.9046310782432556, mass=0.10565999999999999)]


In [ ]:
batch_size=5000

myfilter="/(MissingET.(MET|Phi)|Particle.(PID|Status)|Electron.(PT|Eta|Phi)|Muon.(PT|Eta|Phi)|Jet.(PT|Eta|Phi))/"

#loop over VH/WWW
ntotal_VH=0
nselected_VH=0
ntotal_WWW=0
nselected_WWW=0
df = None
iterator = uproot.iterate([f+":Delphes" for f in root_signal],step_size=batch_size, filter_name=myfilter)
for batch in tqdm(iterator, total=N_root_sig//batch_size):
        higgs=checkHiggs(batch)
        presel=preselection(batch)
        met, metPhi = get_MET(batch)
        
        dr, PT_l0, PT_l1, PT_l2, invarM_l0l1, invarM_l0l2, invarM_l1l2, invarM, \
                transM_l0l1, transM_l0l2, transM_l1l2, \
                deltaeta_l0l1, deltaeta_l0l2, deltaeta_l1l2, \
                deltaangle_l0l1, deltaangle_l0l2, deltaangle_l1l2, \
                dR_l0l1, dR_l0l2, dR_l1l2, totalP = get_minDRll(batch)  #get data (can't be dict due to numba)
        
        dataset = {'PT_l0': PT_l0, 'PT_l1': PT_l1, 'PT_l2': PT_l2, 'met': met, 'metPhi': metPhi,
               'm_l0l1': invarM_l0l1, 'm_l0l2': invarM_l0l2, 'm_l1l2': invarM_l1l2, 'm_lll': invarM,
               'mT_l0l1': transM_l0l1, 'mT_l0l2': transM_l0l2, 'mT_l1l2': transM_l1l2, 
               'delEta_l0l1': deltaeta_l0l1, 'delEta_l0l2': deltaeta_l0l2, 'delEta_l1l2': deltaeta_l1l2,
              'delPhi_l0l1': deltaangle_l0l1, 'delPhi_l0l2': deltaangle_l0l2, 'delPhi_l1l2': deltaangle_l1l2,
              'delR_l0l1': dR_l0l1, 'delR_l0l2': dR_l0l2, 'delR_l1l2': dR_l1l2, 
               'DRll': dr, 'sumPT': totalP} #make into dict
        
        if df is not None:
            newdata = pd.DataFrame(dataset)
            newdata_VH = pd.DataFrame(columns = newdata.keys())
            newdata_WWW = pd.DataFrame(columns = newdata.keys())
            for i in range(0, len(newdata)):
                if np.logical_and(higgs,presel)[i] == True:
                    newdata_VH.loc[df.index[i]] = df.iloc[i] 
                elif np.logical_and(~higgs,presel)[i] == True:
                    newdata_WWW.loc[df.index[i]] = df.iloc[i] 
            df_VH = pd.concat([df_VH, newdata_VH], axis = 0, ignore_index=True)
            df_VH = pd.concat([df_WWW, newdata_WWW], axis = 0, ignore_index=True)
            df = pd.concat([df, newdata], axis = 0, ignore_index=True)
            
        else:
            df = pd.DataFrame(dataset)
            df_VH = pd.DataFrame(columns = df.keys())
            df_WWW = pd.DataFrame(columns = df.keys())
            for i in range(0, len(df)):
                if np.logical_and(higgs,presel)[i] == True:
                    df_VH.loc[df.index[i]] = df.iloc[i]
                elif np.logical_and(~higgs,presel)[i] == True:
                    df_WWW.loc[df.index[i]] = df.iloc[i]
        
        
        #df.loc[df['DRll'][np.logical_and(higgs,presel)] == True, 'type'] = 1
        ntotal_VH+=higgs.sum()
        ntotal_WWW+=(~higgs).sum()
        nselected_VH+=np.logical_and(higgs,presel).sum()
        nselected_WWW+=np.logical_and(~higgs,presel).sum()
                  
print("Preselection efficiency for VH events: %2.2f %%"%(100.*nselected_VH/ntotal_VH))
print("Preselection efficiency for WWW events: %2.2f %%"%(100.*nselected_WWW/ntotal_WWW))


In [67]:
df_VH.insert(loc=0, column='Type', value=2)
#df_WWW.insert(loc=0, column='Type', value=1)

AttributeError: 'NoneType' object has no attribute 'insert'

In [ ]:
#loop over background
ntotal=0
nselected=0
df_bkg_full = None
iterator = uproot.iterate([f+":Delphes" for f in root_bkg],step_size=batch_size, filter_name=myfilter)
for batch in tqdm(iterator, total=N_root_bgk/batch_size):
        higgs=checkHiggs(batch)
        presel=preselection(batch)

        met, metPhi = get_MET(batch)
        dr, PT_l0, PT_l1, PT_l2, invarM_l0l1, invarM_l0l2, invarM_l1l2, invarM, \
                transM_l0l1, transM_l0l2, transM_l1l2, \
                deltaeta_l0l1, deltaeta_l0l2, deltaeta_l1l2, \
                deltaangle_l0l1, deltaangle_l0l2, deltaangle_l1l2, \
                dR_l0l1, dR_l0l2, dR_l1l2, totalP = get_minDRll(batch) #get data (can't be dict due to numba)
        
        dataset = {'PT_l0': PT_l0, 'PT_l1': PT_l1, 'PT_l2': PT_l2, 'met': met, 'metPhi': metPhi,
               'm_l0l1': invarM_l0l1, 'm_l0l2': invarM_l0l2, 'm_l1l2': invarM_l1l2, 'm_lll': invarM,
               'mT_l0l1': transM_l0l1, 'mT_l0l2': transM_l0l2, 'mT_l1l2': transM_l1l2, 
               'delEta_l0l1': deltaeta_l0l1, 'delEta_l0l2': deltaeta_l0l2, 'delEta_l1l2': deltaeta_l1l2,
              'delPhi_l0l1': deltaangle_l0l1, 'delPhi_l0l2': deltaangle_l0l2, 'delPhi_l1l2': deltaangle_l1l2,
              'delR_l0l1': dR_l0l1, 'delR_l0l2': dR_l0l2, 'delR_l1l2': dR_l1l2, 
               'DRll': dr, 'sumPT': totalP} #make into dict
        
        #any round but first round
        if df_bkg_full is not None:
            newdata = pd.DataFrame(dataset)
            newdata_bkg = pd.DataFrame(columns = newdata.keys())
            for i in range(0, len(newdata)):
                if np.logical_and(higgs,presel)[i] == True:
                    newdata_bkg.loc[df_bkg_full.index[i]] = df_bkg_full.iloc[i] 
            df_bkg = pd.concat([df_bkg, newdata_bkg], axis = 0, ignore_index=True)
            df_bkg_full = pd.concat([df_bkg_full, newdata], axis = 0, ignore_index=True)
            
        #first round    
        else:
            df_bkg_full = pd.DataFrame(dataset)
            df_bkg = pd.DataFrame(columns = df.keys())
            for i in range(0, len(df_bkg_full)):
                if presel[i] == True:
                    df_bkg.loc[df_bkg_full.index[i]] = df_bkg_full.iloc[i]
        
        ntotal+=len(batch)
        nselected+=presel.sum()
print("Preselection efficiency for background events: %2.2f %%"%(100.*nselected/ntotal))

In [ ]:
df_bkg.insert(loc=0, column='Type', value=0)
df_bkg.head()

In [ ]:
df_p=pd.concat([df_WWW, df_VH, df_bkg], axis=0, ignore_index=True)
df_p.shape

In [ ]:
corr_df = df_p.corr()

corr_df.dropna()

sns.heatmap(corr_df,
           xticklabels = corr_df.columns,
           yticklabels=corr_df.columns,
           )

In [ ]:
#scale to given luminosity
lumi=400 #inverse fb

n_sig=lumi*xSection_sig*1000
n_bkg=lumi*xSection_bkg*1000

weight_sig=n_sig/N_root_sig
weight_bkg=n_bkg/N_root_bgk

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(12,6))
bins=np.linspace(0.,500.,100)
ax[0].hist([df_VH['met'],df_WWW['met'],df_bkg['met']],bins=bins,
        weights=[np.full(df_VH['met'].shape,weight_sig),
                 np.full(df_WWW['met'].shape,weight_sig),
                 np.full(df_bkg['met'].shape,weight_bkg)],
        histtype='bar', stacked=True,label=["VH","WWW","bkg"])
ax[0].set_yscale('log')
ax[0].text(0.55,0.95,"${\\cal L}=%3.0f$/fb"%lumi,transform=ax[0].transAxes)
ax[0].set_xlabel("MET [GeV]")
ax[0].set_ylabel("Entries")
ax[0].legend()

bins=np.linspace(0.,4.,80)
ax[1].hist([df_VH['DRll'],df_WWW['DRll'],df_bkg['DRll']],bins=bins,
        weights=[np.full(df_VH['DRll'].shape,weight_sig),
                 np.full(df_WWW['DRll'].shape,weight_sig),
                 np.full(df_bkg['DRll'].shape,weight_bkg)],
        histtype='step', stacked=False,density=True,label=["VH","WWW","bkg"])
ax[1].set_yscale('log')
ax[1].text(0.55,0.95,"${\\cal L}=%3.0f$/fb"%lumi,transform=ax[1].transAxes)
ax[1].set_xlabel("min($\\Delta R_{\\ell\\ell})$")
ax[1].set_ylabel("1/N dN/d min($\\Delta R_{\\ell\\ell})$")
ax[1].legend()